In [166]:
"""This file is a test playground for new scripts and functions."""

%load_ext autoreload
%autoreload 2

###### USER SETTINGS ######
SERIAL_ACTIVE = False # if False, just runs the algorithm without sending to HV switches

###### INITIALIZATIONS ######
import cv2
import time
import serial
import numpy as np
import haptic_utils.haptic_map as haptic_map # my custom file
import haptic_utils.USB as USB # my custom file
import haptic_utils.generator as generator
import haptic_utils.algo_gesture as gesture
import matplotlib.pyplot as plt

###### MAIN ######

# initialize camera and gesture model:
frame_rate = 24

output_list = []
output_list.extend(generator.ramp(direction=1))
output_list.extend(generator.ramp(direction=-1))

duty_array_list, period_array_list = haptic_map.linear_map_sequence(output_list,
                                                           freq_range=(0,200),
                                                           duty_range=(0.05,0.5))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
gesture_obj = gesture.Gesture()

In [169]:
output = gesture_obj.output_dict['Thumb_Up'].copy()

In [247]:
len(output.intensity_sequence)

0

In [257]:
output.pop()

(array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]),
 [b'P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00T\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b'P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00T\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b'P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00T\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'])

In [276]:
###### USER SETTINGS ######
SERIAL_ACTIVE = False # if False, just runs the algorithm without sending to HV switches
COM_A = "COM15" # port for MINI switches 1-10
COM_B = "COM9" # port for MINI switches 11-20
COM_C = "COM16" # port for MINI swiches 21-28

###### INITIALIZATIONS ######
import cv2
import time
import haptic_utils.haptic_map as haptic_map
import haptic_utils.generator as generator
import haptic_utils.USB as USB

In [277]:
output_data1 = haptic_map.make_output_data(generator.sine_global(freq=1),
                                                           freq_range=(0,50),
                                                           duty_range=(0.05,0.5))

In [278]:
output_data1.intensity_sequence

[array([[0.37059048, 0.37059048, 0.37059048, 0.37059048, 0.37059048,
         0.37059048, 0.37059048],
        [0.37059048, 0.37059048, 0.37059048, 0.37059048, 0.37059048,
         0.37059048, 0.37059048],
        [0.37059048, 0.37059048, 0.37059048, 0.37059048, 0.37059048,
         0.37059048, 0.37059048],
        [0.37059048, 0.37059048, 0.37059048, 0.37059048, 0.37059048,
         0.37059048, 0.37059048]]),
 array([[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]]),
 array([[0.14644661, 0.14644661, 0.14644661, 0.14644661, 0.14644661,
         0.14644661, 0.14644661],
        [0.14644661, 0.14644661, 0.14644661, 0.14644661, 0.14644661,
         0.14644661, 0.14644661],
        [0.14644661, 0.14644661, 0.14644661, 0.14644661, 0.14644661,
         0.14644661, 0.14644661],
        [0.14644661, 0.14644661, 0.14644661, 0.14644661, 0.14644661